# This program connects to Box.com, gets all of the folders and images and stores information into the database.

https://developer.box.com/en/guides/authentication/oauth2/with-sdk/

In [5]:
!pip install boxsdk
!pip install auth
!pip install redis
!pip install mysql.connector
!pip install requests

In [36]:
from boxsdk import OAuth2

import json
#Set the file we want to use for authenticating a Box app
#The json file stores the client_id and client_secret so we don't have it in the code.
# The json file looks like this:
#{
#"client_id":"___the_codes_for_client_id___",
#"client_secret":"___the_codes_for_client_secret___"
#}

oauth_settings_file = 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\box_app_test.json'
with open(oauth_settings_file, "r") as read_file:
    oauth_data = json.load(read_file)
print(oauth_data["client_id"])
print(oauth_data["client_secret"])

oauth = OAuth2(
    client_id=oauth_data["client_id"],
    client_secret=oauth_data["client_secret"]
)

auth_url, csrf_token = oauth.get_authorization_url('https://jeffblackadar.ca')
print("Click on this:")
print(auth_url)
print(csrf_token)
print("Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.")


rvy5o5ic72c00ldi8vpcqmof0t7h6w85
VWJQ1LTracitLw5HfG8Q0TKplBogFEIT
Click on this:
https://account.box.com/api/oauth2/authorize?state=box_csrf_token_kjGfXZI6hL1Pz41k&response_type=code&client_id=rvy5o5ic72c00ldi8vpcqmof0t7h6w85&redirect_uri=https%3A%2F%2Fjeffblackadar.ca
box_csrf_token_kjGfXZI6hL1Pz41k
Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.


In [37]:
from boxsdk import Client

# Make sure that the csrf token you get from the `state` parameter
# in the final redirect URI is the same token you get from the
# get_authorization_url method to protect against CSRF vulnerabilities.
#assert 'THE_CSRF_TOKEN_YOU_GOT' == csrf_token
access_token, refresh_token = oauth.authenticate('y0Ied4ofurECiUjz2fYy0ErCIlKEI1PH')
client = Client(oauth)

In [11]:
def process_subfolder(client, folder_id, folder_name,cursor_insert):
    print("this folder: "+folder_name)
    items = client.folder(folder_id=folder_id).get_items()
    for item in items:
        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
        if(item.type.capitalize()=="Folder"):
            process_subfolder(client, item.id,folder_name+"/"+item.name,cursor_insert)
        if(item.type.capitalize()=="File"):
            #print(item)
            print('File: {0} is named: "{1}" path: {2} '.format(item.id, item.name, folder_name+"/"+item.name))
            
            insert_image = ("INSERT INTO tbl_box_images (`id_box_file`,`id_box_folder`,`folder`,`file_name`,`img_src`,`img_url`) VALUES (%s,%s,%s,%s,%s,%s);")
            
            cursor_insert.execute(insert_image, (item.id, folder_id, folder_name, item.name, folder_name+"/"+item.name, ""))
            mariadb_connection.commit()
    return




In [ ]:
import requests
import shutil
from pathlib import Path
base_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\pompeii\\box'


import mysql.connector as mariadb

mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")

cursor_insert = mariadb_connection.cursor(buffered=True)

process_subfolder(client, '0',"", cursor_insert)
    
mariadb_connection.close()    


In [20]:
mariadb_connection.close()    

In [55]:
# Image hash functions
# import the necessary packages
from imutils import paths
import argparse
import time
import sys
import cv2
import os

# hashing with OpenCV and PythonPython
def dhash(image, hashSize=8):
    # resize the input image, adding a single column (width) so we
    # can compute the horizontal gradient
    resized = cv2.resize(image, (hashSize + 1, hashSize))

    # compute the (relative) horizontal gradient between adjacent
    # column pixels
    diff = resized[:, 1:] > resized[:, :-1]

    # convert the difference image to a hash
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

def dhash(image, hashSize=8):
    # resize the input image, adding a single column (width) so we
    # can compute the horizontal gradient
    resized = cv2.resize(image, (hashSize + 1, hashSize))
 
    # compute the (relative) horizontal gradient between adjacent
    # column pixels
    diff = resized[:, 1:] > resized[:, :-1]
 
    # convert the difference image to a hash
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

In [63]:
import os
import shutil
from pathlib import Path
base_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\pompeii\\box'

def download_process_box_thumbnail(client, id_box, id_box_file, folder, file_name, cursor_update):
    print("this file: "+id_box_file)
    #get a large 320 pixel thumbnail - it's probably faster, it's all we need for image hashing (I think) and will save on bandwidth and storage
    thumbnail = client.file(id_box_file).get_thumbnail(extension='jpg', max_height='320', max_width='320')

    # load the image for the image has later.  Load it now because it takes a bit of time

    img_local_path=base_dir+folder+"/"+file_name
    #print(img_local_path)
    img_file_name = file_name
    img_local_folder = base_dir+folder

    #see if the image is there already
    if not os.path.exists(img_local_path):
        img_file_name = file_name
        img_local_folder = base_dir+folder
        
        if not os.path.exists(img_local_folder):
            #os.mkdir(img_local_folder)
            path = Path(img_local_folder)
            path.mkdir(parents=True,exist_ok=True)
        #print(img_local_folder+'/'+ img_file_name)
        
        # Open the url image, set stream to True, this will return the stream content.
        # Open a local file with wb ( write binary ) permission.
        #local_file = open(img_local_folder+'/'+ img_file_name, 'wb')
        with open(img_local_folder+'/'+ img_file_name,'wb') as f:
            f.write(thumbnail)
        #print(thumbnail)
        del thumbnail
            #image hash
        imageHash = 0
        print("image hash")
        if not os.path.exists(img_local_path):
            print("Error image not loaded for image hash   " + img_local_path)
        else:         
            # load the image from disk
            path = Path(img_local_folder)
            os.chdir(path)
            hash_image = cv2.imread(img_file_name)
 
            # if the image is None then we could not load it from disk (so skip it)
            if not hash_image is None:
                # convert the image to grayscale and compute the hash
                hash_image = cv2.cvtColor(hash_image, cv2.COLOR_BGR2GRAY)
                imageHash = dhash(hash_image)
                print(imageHash)
            else:
                imageHash = 0
                print("image hash - can't find image?")
            update_image = ("UPDATE tbl_box_images SET image_hash = %s WHERE id = %s;")
            # print(update_image)
            print(id_box)
            cursor_update.execute(update_image, (imageHash, id_box))
            mariadb_connection.commit()


#    for item in items:
#        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
#        if(item.type.capitalize()=="Folder"):
#            process_subfolder(client, item.id,folder_name+"/"+item.name,cursor_insert)
#        if(item.type.capitalize()=="File"):
#            #print(item)
#            print('File: {0} is named: "{1}" path: {2} '.format(item.id, item.name, folder_name+"/"+item.name))
#            
#            insert_image = ("INSERT INTO tbl_box_images (`id_box_file`,`id_box_folder`,`folder`,`file_name`,`img_src`,`img_url`) VALUES (%s,%s,%s,%s,%s,%s);")
#            
#            cursor_insert.execute(insert_image, (item.id, folder_id, folder_name, item.name, folder_name+"/"+item.name, ""))
#            mariadb_connection.commit()
#    return

In [ ]:
download_process_box_thumbnail(client,10, "580392170777", "/pinp5/Altars/Altars/Altar 10110_files", "image008d.jpg", cursor_update)

In [65]:
number_of_rows_to_run = 40
start_id = 12
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_inner = mariadb_connection.cursor(buffered=True)
cursor_update = mariadb_connection.cursor(buffered=True)
cursor.execute("SELECT id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src FROM webpage_images.tbl_box_images WHERE id>=%s AND id<%s", (start_id, end_id))
for id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src in cursor:
    #print(id, id_box_file, img_path_match, id_tbl_webpage_images, img_src)
    if(id_tbl_webpage_images is not None):
        if(id_tbl_webpage_images>0) and (img_src.lower().find(".xml")<0) and (img_src.lower().find(".htm")<0) and (img_src.lower().find(".thmx")<0):
            wi_id=0
            print(img_path_match)
            cursor_inner.execute("SELECT id as wi_id FROM webpage_images.tbl_webpage_images WHERE img_path_match = '%s';", (img_path_match))
            if(wi_id>0):
                update_image = ("UPDATE tbl_box_images SET `id_tbl_webpage_images` = %s WHERE id=%s;")
                #print(update_image)
                cursor_update.execute(update_image, (wi_id, id))
                mariadb_connection.commit()
            else:    
                download_process_box_thumbnail(client, id, str(id_box_file), folder, file_name, cursor_update)
    else:
        download_process_box_thumbnail(client, id, str(id_box_file), folder, file_name, cursor_update)
mariadb_connection.close()    

this file: 580409701973
image hash
1124292005475302354
12


DatabaseError: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction

In [61]:
mariadb_connection.close()    